In [ ]:
%pip install -U kubeflow-training

In [6]:
import sys
sys.path.append("../notebooks")  # needed to make kfto_sdk_mnist
from kfto_sdk_mnist import train_func
from kubeflow.training import TrainingClient
from kubernetes import client as c
import time

In [ ]:
# parameters
num_gpus = "${num_gpus}"
openshift_api_url = "${api_url}"
namespace = "${namespace}"
token = "${password}"
training_image= "${training_image}"

In [8]:
api_key = {"authorization": f"Bearer {token}"}
# config = c.Configuration(host=openshift_api_url, api_key=token)
# config.verify_ssl = False
tc = TrainingClient()

In [ ]:
tc.create_job(
   name="pytorch-ddp",
   namespace=namespace,
   train_func=train_func,
   num_workers=1,
   resources_per_worker={"gpu": num_gpus},
   base_image=training_image,
   packages_to_install=["torchvision==0.19.0","minio==7.2.13", "--target=/tmp/lib"],
   env_vars={"PYTHONPATH": "/tmp/lib:$PYTHONPATH", "NCCL_DEBUG": "INFO", "TORCH_DISTRIBUTED_DEBUG": "DETAIL"}
)

In [ ]:
while not tc.is_job_succeeded(name="pytorch-ddp", namespace=namespace): 
    time.sleep(1)

In [ ]:
time.sleep(10)